In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
method_algo = 'mfcc'
model_algo = 'lstm'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # 1st hidden layer
        keras.layers.LSTM(512, input_shape=[inputs.shape[1], inputs.shape[2]], return_sequences=True),
        keras.layers.LSTM(256),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                            targets_train,
                                                                            test_size=0.2,
                                                                            stratify=targets_train,
                                                                            random_state=random_state)


In [5]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [6]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [8]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
                    validation_data=(inputs_valid, targets_valid),
                    epochs=10000,
                    batch_size=128,
                    callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 87s 923ms/step - loss: 4.6973 - accuracy: 0.0110 - val_loss: 4.6785 - val_accuracy: 0.0184
Epoch 2/10000
89/89 [==============================] - 80s 900ms/step - loss: 4.6158 - accuracy: 0.0320 - val_loss: 4.6183 - val_accuracy: 0.0322
Epoch 3/10000
89/89 [==============================] - 81s 906ms/step - loss: 4.4920 - accuracy: 0.0520 - val_loss: 4.4411 - val_accuracy: 0.0333
Epoch 4/10000
89/89 [==============================] - 80s 895ms/step - loss: 4.2930 - accuracy: 0.0504 - val_loss: 4.2115 - val_accuracy: 0.0524
Epoch 5/10000
89/89 [==============================] - 80s 904ms/step - loss: 4.0338 - accuracy: 0.0751 - val_loss: 3.9647 - val_accuracy: 0.0707
Epoch 6/10000
89/89 [==============================] - 80s 901ms/step - loss: 3.7483 - accuracy: 0.1076 - val_loss: 3.7194 - val_accuracy: 0.1111
Epoch 7/10000
89/89 [==============================] - 80s 902ms/step - loss: 3.4443 - accuracy: 0.1548 - val_loss: 3.4948 -

Epoch 57/10000
89/89 [==============================] - 80s 895ms/step - loss: 0.1210 - accuracy: 0.9745 - val_loss: 2.3298 - val_accuracy: 0.4687
Epoch 58/10000
89/89 [==============================] - 80s 900ms/step - loss: 0.1237 - accuracy: 0.9713 - val_loss: 2.3105 - val_accuracy: 0.4634
Epoch 59/10000
89/89 [==============================] - 80s 898ms/step - loss: 0.1236 - accuracy: 0.9734 - val_loss: 2.2467 - val_accuracy: 0.4715
Epoch 60/10000
89/89 [==============================] - 80s 901ms/step - loss: 0.1217 - accuracy: 0.9724 - val_loss: 2.2762 - val_accuracy: 0.4683
Epoch 61/10000
89/89 [==============================] - 80s 900ms/step - loss: 0.1036 - accuracy: 0.9803 - val_loss: 2.2494 - val_accuracy: 0.4867
Epoch 62/10000
89/89 [==============================] - 80s 899ms/step - loss: 0.0977 - accuracy: 0.9817 - val_loss: 2.2530 - val_accuracy: 0.4797
Epoch 63/10000
89/89 [==============================] - 79s 894ms/step - loss: 0.0911 - accuracy: 0.9827 - val_loss: 2

89/89 [==============================] - 79s 891ms/step - loss: 0.0368 - accuracy: 0.9924 - val_loss: 2.1439 - val_accuracy: 0.5490
Epoch 113/10000
89/89 [==============================] - 80s 902ms/step - loss: 0.0411 - accuracy: 0.9910 - val_loss: 2.2122 - val_accuracy: 0.5363
Epoch 114/10000
89/89 [==============================] - 80s 898ms/step - loss: 0.0428 - accuracy: 0.9880 - val_loss: 2.1718 - val_accuracy: 0.5525
Epoch 115/10000
89/89 [==============================] - 79s 889ms/step - loss: 0.0346 - accuracy: 0.9935 - val_loss: 2.1769 - val_accuracy: 0.5536
Epoch 116/10000
89/89 [==============================] - 80s 897ms/step - loss: 0.0332 - accuracy: 0.9932 - val_loss: 2.1617 - val_accuracy: 0.5585
Epoch 117/10000
89/89 [==============================] - 80s 900ms/step - loss: 0.0371 - accuracy: 0.9917 - val_loss: 2.1399 - val_accuracy: 0.5617
Epoch 118/10000
89/89 [==============================] - 79s 894ms/step - loss: 0.0396 - accuracy: 0.9905 - val_loss: 2.1090 - v

89/89 [==============================] - 43s 480ms/step - loss: 0.0331 - accuracy: 0.9919 - val_loss: 1.9897 - val_accuracy: 0.6013
Epoch 168/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0370 - accuracy: 0.9900 - val_loss: 1.9775 - val_accuracy: 0.6038
Epoch 169/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0300 - accuracy: 0.9937 - val_loss: 1.9945 - val_accuracy: 0.6042
Epoch 170/10000
89/89 [==============================] - 42s 478ms/step - loss: 0.0309 - accuracy: 0.9920 - val_loss: 1.9418 - val_accuracy: 0.6098
Epoch 171/10000
89/89 [==============================] - 42s 478ms/step - loss: 0.0255 - accuracy: 0.9929 - val_loss: 1.9166 - val_accuracy: 0.6116
Epoch 172/10000
89/89 [==============================] - 43s 482ms/step - loss: 0.0227 - accuracy: 0.9952 - val_loss: 1.9495 - val_accuracy: 0.6176
Epoch 173/10000
89/89 [==============================] - 43s 480ms/step - loss: 0.0216 - accuracy: 0.9949 - val_loss: 1.9200 - v

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



89/89 [==============================] - 51s 570ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 1.2339 - val_accuracy: 0.8267
Epoch 1675/10000
89/89 [==============================] - 41s 459ms/step - loss: 5.1160e-04 - accuracy: 0.9999 - val_loss: 1.3256 - val_accuracy: 0.8214
Epoch 1676/10000
89/89 [==============================] - 41s 459ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 1.2880 - val_accuracy: 0.8256
Epoch 1677/10000
89/89 [==============================] - 41s 458ms/step - loss: 9.6987e-04 - accuracy: 0.9998 - val_loss: 1.3022 - val_accuracy: 0.8224
Epoch 1678/10000
89/89 [==============================] - 41s 459ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 1.3100 - val_accuracy: 0.8228
Epoch 1679/10000
89/89 [==============================] - 41s 458ms/step - loss: 0.0036 - accuracy: 0.9988 - val_loss: 1.3107 - val_accuracy: 0.8221
Epoch 1680/10000
89/89 [==============================] - 41s 466ms/step - loss: 0.0031 - accuracy: 0.9989 - val_lo

89/89 [==============================] - 41s 457ms/step - loss: 0.0055 - accuracy: 0.9990 - val_loss: 1.2875 - val_accuracy: 0.8200
Epoch 1729/10000
89/89 [==============================] - 41s 458ms/step - loss: 0.0026 - accuracy: 0.9992 - val_loss: 1.2967 - val_accuracy: 0.8171
Epoch 1730/10000
89/89 [==============================] - 41s 456ms/step - loss: 0.0022 - accuracy: 0.9991 - val_loss: 1.2334 - val_accuracy: 0.8362
Epoch 1731/10000
89/89 [==============================] - 41s 458ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 1.2336 - val_accuracy: 0.8359
Epoch 1732/10000
89/89 [==============================] - 41s 458ms/step - loss: 0.0027 - accuracy: 0.9990 - val_loss: 1.3210 - val_accuracy: 0.8238
Epoch 1733/10000
89/89 [==============================] - 41s 458ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 1.2524 - val_accuracy: 0.8330
Epoch 1734/10000
89/89 [==============================] - 41s 459ms/step - loss: 6.7965e-04 - accuracy: 0.9999 - val_loss: 

89/89 [==============================] - 41s 456ms/step - loss: 2.7851e-04 - accuracy: 1.0000 - val_loss: 1.2942 - val_accuracy: 0.8281
Epoch 1783/10000
89/89 [==============================] - 41s 457ms/step - loss: 3.8573e-04 - accuracy: 1.0000 - val_loss: 1.3001 - val_accuracy: 0.8313
Epoch 1784/10000
89/89 [==============================] - 41s 457ms/step - loss: 4.9502e-04 - accuracy: 0.9998 - val_loss: 1.3124 - val_accuracy: 0.8341
Epoch 1785/10000
89/89 [==============================] - 41s 457ms/step - loss: 2.6327e-04 - accuracy: 1.0000 - val_loss: 1.3063 - val_accuracy: 0.8348
Epoch 1786/10000
89/89 [==============================] - 41s 457ms/step - loss: 4.0182e-04 - accuracy: 1.0000 - val_loss: 1.2781 - val_accuracy: 0.8320
Epoch 1787/10000
89/89 [==============================] - 41s 457ms/step - loss: 8.5100e-04 - accuracy: 0.9998 - val_loss: 1.3796 - val_accuracy: 0.8228
Epoch 1788/10000
89/89 [==============================] - 41s 458ms/step - loss: 0.0018 - accuracy:

Epoch 1836/10000
89/89 [==============================] - 41s 457ms/step - loss: 4.3230e-04 - accuracy: 1.0000 - val_loss: 1.2179 - val_accuracy: 0.8263
Epoch 1837/10000
89/89 [==============================] - 41s 457ms/step - loss: 4.1428e-04 - accuracy: 0.9999 - val_loss: 1.2033 - val_accuracy: 0.8302
Epoch 1838/10000
89/89 [==============================] - 41s 457ms/step - loss: 8.6446e-04 - accuracy: 0.9998 - val_loss: 1.2526 - val_accuracy: 0.8281
Epoch 1839/10000
89/89 [==============================] - 41s 458ms/step - loss: 7.1938e-04 - accuracy: 0.9997 - val_loss: 1.2091 - val_accuracy: 0.8288
Epoch 1840/10000
89/89 [==============================] - 40s 454ms/step - loss: 4.1084e-04 - accuracy: 1.0000 - val_loss: 1.2218 - val_accuracy: 0.8355
Epoch 1841/10000
89/89 [==============================] - 40s 454ms/step - loss: 5.2166e-04 - accuracy: 0.9998 - val_loss: 1.2166 - val_accuracy: 0.8334
Epoch 1842/10000
89/89 [==============================] - 41s 457ms/step - loss: 0

In [9]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [10]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

28/28 [==============================] - 5s 161ms/step - loss: 1.2479 - accuracy: 0.8311


In [11]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                           f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')